In [3]:
import os
import sys
path_to_this_notebook = os.path.abspath('.')
PATH_TO_PROJECT = path_to_this_notebook[: path_to_this_notebook.find('notebooks')]
sys.path.append(PATH_TO_PROJECT)
import pandas as pd
import numpy as np
import librosa
from skimage.transform import resize
import matplotlib.pyplot as plt
from collections import defaultdict
import re
from src.metadata_processing.filters import filter_recs_metadata
from src.util import make_image
%matplotlib inline

In [7]:
# load metadata
path_to_recordings_metadata = PATH_TO_PROJECT + 'data/'
recordings_metadata_name = 'recordings_metadata.csv'

# Filters should be either 'all' or [val1, val2, ..., ]

colonies = ['arryn']
#colonies = ['stark', 'martell']
dates = 'all'
experiments = 'all'
stages = ['traced and checked']

# load data
recs_metadata = pd.read_csv(path_to_recordings_metadata + recordings_metadata_name)
filters_mask = filter_recs_metadata(recs_metadata, dates = dates, colonies = colonies, 
                                    stages = stages, experiments = experiments)
recs_to_check = recs_metadata[filters_mask]
print(recs_to_check.shape)
recs_to_check.head(10)

(40, 8)


,colony,date,experiment,name,number,path,processing stage,ratids
6,arryn,24-05-19,single rat softchirps,arryn_24-05-19_9624_0000001.npy,1.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,9624
13,arryn,08-11-19,single rat softchirps,arryn_08-11-19_9435_0000053.npy,53.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,9435
15,arryn,21-06-19,single rat softchirps,arryn_21-06-19_9435_0000001.npy,1.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,9435
18,arryn,24-05-19,single rat softchirps,arryn_24-05-19_0827_0000001.npy,1.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,827
31,arryn,20-04-19,single rat softchirps,arryn_20-04-19_9624_0000001.npy,1.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,9624
35,arryn,24-05-19,single rat softchirps,arryn_24-05-19_0088_0000001.npy,1.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,88
58,arryn,24-05-19,single rat softchirps,arryn_24-05-19_9435_0000001.npy,1.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,9435
65,arryn,15-05-19,single rat softchirps,arryn_15-05-19_9423_0000001.npy,1.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,9423
70,arryn,08-11-19,single rat softchirps,arryn_08-11-19_0088_0000048.npy,48.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,88
72,arryn,15-05-19,single rat softchirps,arryn_15-05-19_9469_0000001.npy,1.0,/home/gr1/Projects/naked-mole-rats/data/arryn/...,traced and checked,9469


In [8]:
recs_metadata = pd.read_csv(path_to_recordings_metadata + recordings_metadata_name)
filters_mask = filter_recs_metadata(recs_metadata, dates = dates, colonies = colonies, 
                                         stages = stages, experiments = experiments)
recs_to_check = recs_metadata[filters_mask]

checked = 0
total_traces_to_check = [t  for rec_path, rec_name in recs_to_check[['path', 'name']].values
                         for t in os.listdir(rec_path.replace('recordings', 'traces'))
                         if (rec_name[:-3] in t) and 'freshtrace' in t]

for rec_path, rec_name in recs_to_check[['path', 'name']].values:
    traces_in_rec = [t for t in os.listdir(rec_path.replace('recordings', 'traces')) 
                     if (rec_name[:-3] in t) and 'freshtrace' in t]
    
    rec_ind = np.where(recs_metadata['name'] == rec_name)[0][0]
    rec_stage = recs_metadata['processing stage'][recs_metadata['name'] == rec_name].iloc[0]
    rec_new_stage = rec_stage + ' and checked'
    rec = np.load(rec_path + rec_name)
    df_txt = pd.read_csv(rec_path + rec_name[:-3] + 'txt', sep = '\t')
    print('This rec contains %d softchirps' % (np.sum(df_txt['cl'] == 'softchirp')))
    print('Checking traces from recording %s' % rec_name)
    print('Type 1 or nothing if trace is good, type anything else to delete it')
    for t_name in traces_in_rec:
        trace = rgb2gray(plt.imread(rec_path.replace('recordings', 'traces') + t_name))
        s, e = map(float, re.findall('[0-9]+\.[0-9]+\_[0-9]+\.[0-9]+', t_name)[0].split('_'))
        s_int = int(22050 * s)
        e_int = int(22050 * e)
        
        sound = rec[s_int : e_int]
        
        plt.subplot(121)
        plt.imshow(make_image(sound))
        plt.subplot(122)
        plt.imshow(trace)
        plt.show()
        inp = input()
        if inp != '1' and inp != '':
            #os.remove(rec_path.replace('recordings', 'traces') + t_name)
            pass
        else:
            #os.rename(rec_path.replace('recordings', 'traces') + t_name, 
            #rec_path.replace('recordings', 'traces') + t_name.replace('fresh', ''))
            pass   
        
        checked += 1
    print('Checked %d/%d' %(checked, len(total_traces_to_check)))
    recs_metadata['processing stage'].iloc[rec_ind] = rec_new_stage
    print('Saved')
    recs_metadata.to_csv(path_to_recordings_metadata + recordings_metadata_name, index =  None)


This rec contains 12 softchirps
Checking traces from recording arryn_24-05-19_9624_0000001.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 23 softchirps
Checking traces from recording arryn_08-11-19_9435_0000053.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 31 softchirps
Checking traces from recording arryn_21-06-19_9435_0000001.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 28 softchirps
Checking traces from recording arryn_24-05-19_0827_0000001.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 15 softchirps
Checking traces from recording arryn_20-04-19_9624_0000001.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 23 softchirps
Checking traces from recording arryn_24-05-19_0088_0000001.npy


/home/gr1/miniconda3/envs/nmr/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


This rec contains 9 softchirps
Checking traces from recording arryn_08-11-19_9624_0000049.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 13 softchirps
Checking traces from recording arryn_21-06-19_9423_0000001.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 24 softchirps
Checking traces from recording arryn_20-04-19_9435_0000001.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 25 softchirps
Checking traces from recording arryn_21-06-19_9469_0000001.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 40 softchirps
Checking traces from recording arryn_15-05-19_9442_0000001.npy
Type 1 or nothing if trace is good, type anything else to delete it
Checked 0/0
Saved
This rec contains 26 softchirps
Checking traces from recording arryn_21-06-19_0827_0000001.npy
T